In [ ]:
!pip install requests beautifulsoup4 selenium webdriver_manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
pip install selenium mysql-connector-python nltk pillow pytesseract


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mahbu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahbu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
Childhood cancer immunotherapy success rate


**search term frequency**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import mysql.connector
from urllib.parse import urlparse
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
from PIL import Image
import pytesseract
import hashlib

# Function to perform search and extract URLs and titles
def search_and_extract_urls(search_term, search_engine):
    chromedriver_path = "C:\\chromedriver-win64\\chromedriver.exe"  # Adjust this path accordingly
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service)

    if search_engine.lower() == "google":
        driver.get("https://www.google.com/")
        search_box_name = "q"
    elif search_engine.lower() == "bing":
        driver.get("https://www.bing.com/")
        search_box_name = "q"
    elif search_engine.lower() == "yahoo":
        driver.get("https://search.yahoo.com/")
        search_box_name = "p"
    elif search_engine.lower() == "duckduckgo":
        driver.get("https://duckduckgo.com/")
        search_box_name = "q"
    else:
        print("Invalid search engine specified.")
        driver.quit()
        return

    search_box = driver.find_element(By.NAME, search_box_name)
    search_box.send_keys(search_term)
    search_box.send_keys(Keys.RETURN)

    unique_urls = set()
    url_titles = []
    max_iterations = 10
    current_iteration = 0
    while current_iteration < max_iterations:
        links = driver.find_elements(By.CSS_SELECTOR, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href and not href.startswith("https://www.google.com/search"):
                parsed_url = urlparse(href)
                base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
                if is_relevant(base_url, search_term):
                    unique_urls.add(base_url)
                    title = link.get_attribute('textContent')
                    url_titles.append((base_url, title))
        current_iteration += 1
        if search_engine.lower() == "yahoo":
            try:
                next_button = driver.find_element(By.XPATH, "//a[@class='next']")
                if next_button:
                    next_button.click()
                else:
                    break
            except:
                break
        else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

    screenshot_url = save_screenshot(driver, search_engine)
    driver.quit()

    return url_titles, screenshot_url

def is_relevant(url, search_term):
    tokens = word_tokenize(search_term.lower())
    stop_words = set(stopwords.words('english'))
    keywords = [word for word in tokens if word.isalnum() and word not in stop_words]

    for keyword in keywords:
        if keyword in url.lower():
            return True
    return False

def is_advertisement(url):
    advertisement_keywords = ["ad", "sponsored", "promotion"]
    for keyword in advertisement_keywords:
        if keyword in url.lower():
            return True
    return False

def save_search_term(search_term):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        query = "SELECT term_id FROM SearchTerms WHERE search_term = %s"
        cursor.execute(query, (search_term,))
        existing_term = cursor.fetchone()

        if existing_term:
            term_id = existing_term[0]
        else:
            insert_query = "INSERT INTO SearchTerms (search_term) VALUES (%s)"
            cursor.execute(insert_query, (search_term,))
            term_id = cursor.lastrowid
            print("Search term saved successfully.")

        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

    return term_id

def save_search_result(term_id, search_engine, url, title, screenshot_url, search_term):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor(buffered=True)

    try:
        url_hash = hashlib.sha256(url.encode()).hexdigest()

        select_query = "SELECT result_id FROM SearchResults WHERE term_id = %s AND search_engine = %s AND url_hash = %s"
        cursor.execute(select_query, (term_id, search_engine, url_hash))
        existing_result = cursor.fetchone()

        if existing_result:
            result_id = existing_result[0]
        else:
            insert_query = "INSERT INTO SearchResults (term_id, search_engine, url, url_hash, title, screenshot) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(insert_query, (term_id, search_engine, url, url_hash, title, screenshot_url))
            result_id = cursor.lastrowid

        select_frequency_query = "SELECT frequency FROM UrlFrequency WHERE term_id = %s AND url_hash = %s"
        cursor.execute(select_frequency_query, (term_id, url_hash))
        existing_frequency = cursor.fetchone()

        if existing_frequency:
            update_frequency_query = "UPDATE UrlFrequency SET frequency = frequency + 1 WHERE term_id = %s AND url_hash = %s"
            cursor.execute(update_frequency_query, (term_id, url_hash))
        else:
            insert_frequency_query = "INSERT INTO UrlFrequency (term_id, url, url_hash, frequency) VALUES (%s, %s, %s, 1)"
            cursor.execute(insert_frequency_query, (term_id, url, url_hash))

        connection.commit()

        # Save token frequencies
        tokens = tokenize_text(title)
        for token, frequency in tokens.items():
            save_token_frequency(result_id, token, frequency)

        # Count the number of search term tokens found in the title
        search_term_tokens = tokenize_text(search_term)
        token_count = {token: title.lower().count(token) for token in search_term_tokens}
        save_term_token_frequency(term_id, token_count)

        print("Search result saved successfully.")
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

    return term_id

def save_token_frequency(result_id, token, frequency):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        insert_query = "INSERT INTO TokenFrequencies (result_id, token, frequency) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (result_id, token, frequency))
        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

def save_term_token_frequency(term_id, token_count):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        for token, count in token_count.items():
            select_query = "SELECT frequency FROM SearchTermFrequencies WHERE term_id = %s AND token = %s"
            cursor.execute(select_query, (term_id, token))
            existing_frequency = cursor.fetchone()

            if existing_frequency:
                update_query = "UPDATE SearchTermFrequencies SET frequency = frequency + %s WHERE term_id = %s AND token = %s"
                cursor.execute(update_query, (count, term_id, token))
            else:
                insert_query = "INSERT INTO SearchTermFrequencies (term_id, token, frequency) VALUES (%s, %s, %s)"
                cursor.execute(insert_query, (term_id, token, count))

        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    token_count = {}
    for token in filtered_tokens:
        if token in token_count:
            token_count[token] += 1
        else:
            token_count[token] = 1
    return token_count

def save_screenshot(driver, search_engine):
    try:
        filename = f"{search_engine}_results.png"
        screenshot_path = os.path.join(screenshot_dir, filename)
        driver.save_screenshot(screenshot_path)
        screenshot_text = ocr_text_from_image(screenshot_path)
        print(f"Screenshot saved successfully: {filename}")
        return screenshot_path
    except Exception as e:
        print(f"Error occurred while taking screenshot for {search_engine}: {e}")

def ocr_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

def main():
    search_term = input("Enter search term: ")
    term_id = save_search_term(search_term)
    search_engines = ["google", "bing", "yahoo", "duckduckgo"]

    for search_engine in search_engines:
        search_results, screenshot_url = search_and_extract_urls(search_term, search_engine)
        for url, title in search_results:
            save_search_result(term_id, search_engine, url, title, screenshot_url, search_term)
        ad_free_results = [url for url, title in search_results if not is_advertisement(url)]
        print(f"Extracted URLs from {search_engine}:")
        for url in ad_free_results:
            print(url)

if __name__ == "__main__":
    screenshot_dir = "C:\\Users\\mahbu\\screenshots"
    if not os.path.exists(screenshot_dir):
        os.makedirs(screenshot_dir)
    main()


Enter search term: Childhood cancer immunotherapy success rate
Screenshot saved successfully: google_results.png
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Search result saved successfully.
Sea

In [ ]:
# Importing necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import mysql.connector
from urllib.parse import urlparse
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
from PIL import Image
import pytesseract
import hashlib

# Function to perform search and extract URLs and titles
def search_and_extract_urls(search_term, search_engine):
    # Setting up Selenium WebDriver
    chromedriver_path = "C:\\chromedriver-win64\\chromedriver.exe"  # Path to ChromeDriver
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service)

    # Opening the specified search engine
    if search_engine.lower() == "google":
        driver.get("https://www.google.com/")
        search_box_name = "q"
    elif search_engine.lower() == "bing":
        driver.get("https://www.bing.com/")
        search_box_name = "q"
    elif search_engine.lower() == "yahoo":
        driver.get("https://search.yahoo.com/")
        search_box_name = "p"
    elif search_engine.lower() == "duckduckgo":
        driver.get("https://duckduckgo.com/")
        search_box_name = "q"
    else:
        print("Invalid search engine specified.")
        driver.quit()
        return

    # Entering the search term into the search box
    search_box = driver.find_element(By.NAME, search_box_name)
    search_box.send_keys(search_term)
    search_box.send_keys(Keys.RETURN)

    # Extracting unique URLs and their titles from the search results
    unique_urls = set()
    url_titles = []
    max_iterations = 10  # Maximum iterations to scroll or navigate through pages
    current_iteration = 0
    while current_iteration < max_iterations:
        links = driver.find_elements(By.CSS_SELECTOR, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href and not href.startswith("https://www.google.com/search"):
                parsed_url = urlparse(href)
                base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
                if is_relevant(base_url, search_term):
                    unique_urls.add(base_url)
                    title = link.get_attribute('textContent')
                    url_titles.append((base_url, title))
        current_iteration += 1
        if search_engine.lower() == "yahoo":
            try:
                next_button = driver.find_element(By.XPATH, "//a[@class='next']")
                if next_button:
                    next_button.click()
                else:
                    break
            except:
                break
        else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for 2 seconds after scrolling

    # Saving a screenshot of the search results page
    screenshot_url = save_screenshot(driver, search_engine)
    driver.quit()

    return url_titles, screenshot_url

# Function to check if a URL is relevant to the search term
def is_relevant(url, search_term):
    tokens = word_tokenize(search_term.lower())
    stop_words = set(stopwords.words('english'))
    keywords = [word for word in tokens if word.isalnum() and word not in stop_words]

    for keyword in keywords:
        if keyword in url.lower():
            return True
    return False

# Function to check if a URL is an advertisement
def is_advertisement(url):
    advertisement_keywords = ["ad", "sponsored", "promotion"]
    for keyword in advertisement_keywords:
        if keyword in url.lower():
            return True
    return False

# Function to save the search term into the database
def save_search_term(search_term):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        query = "SELECT term_id FROM SearchTerms WHERE search_term = %s"
        cursor.execute(query, (search_term,))
        existing_term = cursor.fetchone()

        if existing_term:
            term_id = existing_term[0]
        else:
            insert_query = "INSERT INTO SearchTerms (search_term) VALUES (%s)"
            cursor.execute(insert_query, (search_term,))
            term_id = cursor.lastrowid
            print("Search term saved successfully.")

        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

    return term_id

# Function to save search result into the database
def save_search_result(term_id, search_engine, url, title, screenshot_url, search_term):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor(buffered=True)

    try:
        # Hashing URL to ensure uniqueness
        url_hash = hashlib.sha256(url.encode()).hexdigest()

        # Checking if the search result already exists in the database
        select_query = "SELECT result_id FROM SearchResults WHERE term_id = %s AND search_engine = %s AND url_hash = %s"
        cursor.execute(select_query, (term_id, search_engine, url_hash))
        existing_result = cursor.fetchone()

        if existing_result:
            result_id = existing_result[0]
        else:
            # Inserting the search result into the database
            insert_query = "INSERT INTO SearchResults (term_id, search_engine, url, url_hash, title, screenshot) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(insert_query, (term_id, search_engine, url, url_hash, title, screenshot_url))
            result_id = cursor.lastrowid

        # Updating or inserting URL frequency into the database
        select_frequency_query = "SELECT frequency FROM UrlFrequency WHERE term_id = %s AND url_hash = %s"
        cursor.execute(select_frequency_query, (term_id, url_hash))
        existing_frequency = cursor.fetchone()

        if existing_frequency:
            update_frequency_query = "UPDATE UrlFrequency SET frequency = frequency + 1 WHERE term_id = %s AND url_hash = %s"
            cursor.execute(update_frequency_query, (term_id, url_hash))
        else:
            insert_frequency_query = "INSERT INTO UrlFrequency (term_id, url, url_hash, frequency) VALUES (%s, %s, %s, 1)"
            cursor.execute(insert_frequency_query, (term_id, url, url_hash))

        connection.commit()

        # Save token frequencies
        tokens = tokenize_text(title)
        for token, frequency in tokens.items():
            save_token_frequency(result_id, token, frequency)

        # Count the number of search term tokens found in the title
        search_term_tokens = tokenize_text(search_term)
        token_count = {token: title.lower().count(token) for token in search_term_tokens}
        save_term_token_frequency(term_id, token_count)

        print("Search result saved successfully.")
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

    return term_id

# Function to save token frequency into the database
def save_token_frequency(result_id, token, frequency):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        insert_query = "INSERT INTO TokenFrequencies (result_id, token, frequency) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (result_id, token, frequency))
        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

# save term token frequency into the database
def save_term_token_frequency(term_id, token_count):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        for token, count in token_count.items():
            select_query = "SELECT frequency FROM SearchTermFrequencies WHERE term_id = %s AND token = %s"
            cursor.execute(select_query, (term_id, token))
            existing_frequency = cursor.fetchone()

            if existing_frequency:
                update_query = "UPDATE SearchTermFrequencies SET frequency = frequency + %s WHERE term_id = %s AND token = %s"
                cursor.execute(update_query, (count, term_id, token))
            else:
                insert_query = "INSERT INTO SearchTermFrequencies (term_id, token, frequency) VALUES (%s, %s, %s)"
                cursor.execute(insert_query, (term_id, token, count))

        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

# Function to tokenize text and count word frequencies
def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    token_count = {}
    for token in filtered_tokens:
        if token in token_count:
            token_count[token] += 1
        else:
            token_count[token] = 1
    return token_count

# Function to save a screenshot and perform OCR on the image
def save_screenshot(driver, search_engine):
    try:
        filename = f"{search_engine}_results.png"
        screenshot_path = os.path.join(screenshot_dir, filename)
        driver.save_screenshot(screenshot_path)
        screenshot_text = ocr_text_from_image(screenshot_path)
        print(f"Screenshot saved successfully: {filename}")
        return screenshot_path
    except Exception as e:
        print(f"Error occurred while taking screenshot for {search_engine}: {e}")

# Function to perform OCR (Optical Character Recognition) on an image
def ocr_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# Main function to execute the program
def main():
    search_term = input("Enter search term: ")
    term_id = save_search_term(search_term)
    search_engines = ["google", "bing", "yahoo", "duckduckgo"]

    for search_engine in search_engines:
        search_results, screenshot_url = search_and_extract_urls(search_term, search_engine)
        for url, title in search_results:
            save_search_result(term_id, search_engine, url, title, screenshot_url, search_term)
        ad_free_results = [url for url, title in search_results if not is_advertisement(url)]
        print(f"Extracted URLs from {search_engine}:")
        for url in ad_free_results:
            print(url)

# Entry point of the script
if __name__ == "__main__":
    # Creating a directory for storing screenshots if it doesn't exist
    screenshot_dir = "C:\\Users\\mahbu\\screenshots"
    if not os.path.exists(screenshot_dir):
        os.makedirs(screenshot_dir)
    # Calling the main function to start the program
    main()


In [ ]:
# Importing necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import mysql.connector
from urllib.parse import urlparse
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
from PIL import Image
import pytesseract
import hashlib

# Function to perform search and extract URLs and titles
def search_and_extract_urls(search_term, search_engine):
    # Setting up Selenium WebDriver
    chromedriver_path = "C:\\chromedriver-win64\\chromedriver.exe"  # Path to ChromeDriver
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service)

    # Opening the specified search engine
    if search_engine.lower() == "google":
        driver.get("https://www.google.com/")
        search_box_name = "q"
    elif search_engine.lower() == "bing":
        driver.get("https://www.bing.com/")
        search_box_name = "q"
    elif search_engine.lower() == "yahoo":
        driver.get("https://search.yahoo.com/")
        search_box_name = "p"
    elif search_engine.lower() == "duckduckgo":
        driver.get("https://duckduckgo.com/")
        search_box_name = "q"
    else:
        print("Invalid search engine specified.")
        driver.quit()
        return

    # Entering the search term into the search box
    search_box = driver.find_element(By.NAME, search_box_name)
    search_box.send_keys(search_term)
    search_box.send_keys(Keys.RETURN)

    # Extracting unique URLs and their titles from the search results
    unique_urls = set()
    url_titles = []
    max_iterations = 10  # Maximum iterations to scroll or navigate through pages
    current_iteration = 0
    while current_iteration < max_iterations:
        links = driver.find_elements(By.CSS_SELECTOR, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href and not href.startswith("https://www.google.com/search"):
                parsed_url = urlparse(href)
                base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
                if is_relevant(base_url, search_term):
                    unique_urls.add(base_url)
                    title = link.get_attribute('textContent')
                    url_titles.append((base_url, title))
        current_iteration += 1
        if search_engine.lower() == "yahoo":
            try:
                next_button = driver.find_element(By.XPATH, "//a[@class='next']")
                if next_button:
                    next_button.click()
                else:
                    break
            except:
                break
        else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for 2 seconds after scrolling

    # Saving a screenshot of the search results page
    screenshot_url = save_screenshot(driver, search_engine)
    driver.quit()

    return url_titles, screenshot_url

# Function to check if a URL is relevant to the search term
def is_relevant(url, search_term):
    tokens = word_tokenize(search_term.lower())
    stop_words = set(stopwords.words('english'))
    keywords = [word for word in tokens if word.isalnum() and word not in stop_words]

    for keyword in keywords:
        if keyword in url.lower():
            return True
    return False

# Function to check if a URL is an advertisement
def is_advertisement(url):
    advertisement_keywords = ["ad", "sponsored", "promotion"]
    for keyword in advertisement_keywords:
        if keyword in url.lower():
            return True
    return False

# Function to save the search term into the database
def save_search_term(search_term):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        query = "SELECT term_id FROM SearchTerms WHERE search_term = %s"
        cursor.execute(query, (search_term,))
        existing_term = cursor.fetchone()

        if existing_term:
            term_id = existing_term[0]
        else:
            insert_query = "INSERT INTO SearchTerms (search_term) VALUES (%s)"
            cursor.execute(insert_query, (search_term,))
            term_id = cursor.lastrowid
            print("Search term saved successfully.")

        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

    return term_id

# Function to save search result into the database
def save_search_result(term_id, search_engine, url, title, screenshot_url, search_term):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor(buffered=True)

    try:
        # Hashing URL to ensure uniqueness
        url_hash = hashlib.sha256(url.encode()).hexdigest()

        # Checking if the search result already exists in the database
        select_query = "SELECT result_id FROM SearchResults WHERE term_id = %s AND search_engine = %s AND url_hash = %s"
        cursor.execute(select_query, (term_id, search_engine, url_hash))
        existing_result = cursor.fetchone()

        if existing_result:
            result_id = existing_result[0]
        else:
            # Inserting the search result into the database
            insert_query = "INSERT INTO SearchResults (term_id, search_engine, url, url_hash, title, screenshot) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(insert_query, (term_id, search_engine, url, url_hash, title, screenshot_url))
            result_id = cursor.lastrowid

        # Updating or inserting URL frequency into the database
        select_frequency_query = "SELECT frequency FROM UrlFrequency WHERE term_id = %s AND url_hash = %s"
        cursor.execute(select_frequency_query, (term_id, url_hash))
        existing_frequency = cursor.fetchone()

        if existing_frequency:
            update_frequency_query = "UPDATE UrlFrequency SET frequency = frequency + 1 WHERE term_id = %s AND url_hash = %s"
            cursor.execute(update_frequency_query, (term_id, url_hash))
        else:
            insert_frequency_query = "INSERT INTO UrlFrequency (term_id, url, url_hash, frequency) VALUES (%s, %s, %s, 1)"
            cursor.execute(insert_frequency_query, (term_id, url, url_hash))

        connection.commit()

        # Save token frequencies
        tokens = tokenize_text(title)
        for token, frequency in tokens.items():
            save_token_frequency(result_id, token, frequency)

        # Count the number of search term tokens found in the title
        search_term_tokens = tokenize_text(search_term)
        token_count = {token: title.lower().count(token) for token in search_term_tokens}
        save_term_token_frequency(term_id, token_count)

        print("Search result saved successfully.")
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

    return term_id

# Function to save token frequency into the database
def save_token_frequency(result_id, token, frequency):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        insert_query = "INSERT INTO TokenFrequencies (result_id, token, frequency) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (result_id, token, frequency))
        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

# save term token frequency into the database
def save_term_token_frequency(term_id, token_count):
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Jyoti123",
        database="MY_CUSTOM_BOT",
        auth_plugin="mysql_native_password"
    )
    cursor = connection.cursor()

    try:
        for token, count in token_count.items():
            select_query = "SELECT frequency FROM SearchTermFrequencies WHERE term_id = %s AND token = %s"
            cursor.execute(select_query, (term_id, token))
            existing_frequency = cursor.fetchone()

            if existing_frequency:
                update_query = "UPDATE SearchTermFrequencies SET frequency = frequency + %s WHERE term_id = %s AND token = %s"
                cursor.execute(update_query, (count, term_id, token))
            else:
                insert_query = "INSERT INTO SearchTermFrequencies (term_id, token, frequency) VALUES (%s, %s, %s)"
                cursor.execute(insert_query, (term_id, token, count))

        connection.commit()
    except mysql.connector.Error as error:
        print("Error:", error)
    finally:
        cursor.close()
        connection.close()

# Function to tokenize text and count word frequencies
def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    token_count = {}
    for token in filtered_tokens:
        if token in token_count:
            token_count[token] += 1
        else:
            token_count[token] = 1
    return token_count

# Function to save a screenshot and perform OCR on the image
def save_screenshot(driver, search_engine):
    try:
        filename = f"{search_engine}_results.png"
        screenshot_path = os.path.join(screenshot_dir, filename)
        driver.save_screenshot(screenshot_path)
        screenshot_text = ocr_text_from_image(screenshot_path)
        print(f"Screenshot saved successfully: {filename}")
        return screenshot_path
    except Exception as e:
        print(f"Error occurred while taking screenshot for {search_engine}: {e}")

# Function to perform OCR (Optical Character Recognition) on an image
def ocr_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# Main function to execute the program
def main():
    search_term = input("Enter search term: ")
    term_id = save_search_term(search_term)
    search_engines = ["google", "bing", "yahoo", "duckduckgo"]

    for search_engine in search_engines:
        search_results, screenshot_url = search_and_extract_urls(search_term, search_engine)
        for url, title in search_results:
            save_search_result(term_id, search_engine, url, title, screenshot_url, search_term)
        ad_free_results = [url for url, title in search_results if not is_advertisement(url)]
        print(f"Extracted URLs from {search_engine}:")
        for url in ad_free_results:
            print(url)

# Entry point of the script
if __name__ == "__main__":
    # Creating a directory for storing screenshots if it doesn't exist
    screenshot_dir = "C:\\Users\\mahbu\\screenshots"
    if not os.path.exists(screenshot_dir):
        os.makedirs(screenshot_dir)
    # Calling the main function to start the program
    main()

